# Einlesen des Datensatzes:

In [7]:
import pandas as pd
df = pd.read_json("../yelp_dataset/yelp_dataset~/dataset/business.json", lines=True)

In [8]:
df.columns

Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'neighborhood',
       'postal_code', 'review_count', 'stars', 'state'],
      dtype='object')

In [23]:
len(df)

174567

In [10]:
df.head(3)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ


# Betrachtung der ZIP-Codes

In [26]:
zip_codes = list(set(df.postal_code.values))
len(zip_codes)

16005

In [28]:
num_zip_codes = [code for code in zip_codes if code.isnumeric()]
len(num_zip_codes)

959

In [34]:
len(df.loc[df.postal_code.str.isnumeric()])

131061

In [52]:
alpha_num_zip_codes = [code for code in zip_codes if code.replace(" ", "").isalnum()]

In [53]:
len(alpha_num_zip_codes)

16004

# Crawlen der Race-Statistik nach ZIP-Codes

In [21]:
from lib.crawler import crawl_race_stats

In [44]:
crashcounter = 0
def crawl(zipcode):
    global crashcounter
    try:
        result = crawl_race_stats(zipcode)
        result["postal_code"] = zipcode
        return result
    except Exception as e:
        print(f"For zipcode: {zipcode}, {e}", end="\r")
        crashcounter += 1
     

In [45]:
results = []
for zipcode in num_zip_codes:
    results.append(crawl(str(zipcode)))


In [47]:
res2 = [r for r in results if r]

In [54]:
postal_df = pd.DataFrame(res2)

In [55]:
postal_df.head(5)

,Asian,Black,Hispanic,Mixed,Other,White,postal_code
0,0.4%,64.4%,3.3%,1.1%,0.2%,30.6%,27536
1,3.4%,4.6%,16.3%,3.8%,0.0%,71.8%,85355
2,1.8%,4.1%,0.1%,1.4%,0.0%,92.6%,15046
3,0.0%,1.7%,4.6%,0.7%,1.7%,91.2%,89013
4,4.0%,0.7%,20.9%,1.1%,0.1%,73.1%,60191


In [56]:
postal_df.to_csv("data/race_by_postalcode.csv", sep=",", encoding="utf-8")

In [57]:
len(df[df.postal_code.isin(postal_df.postal_code.values)])

127462

# Überlegungen zu Koordinaten

In [58]:
coords = [(lat, long) for lat, long in zip(df.latitude, df.longitude)]

In [69]:
coords_2dec = [(round(lat, 2), round(long, 2))for lat, long in zip(df.latitude, df.longitude)]
coords_3dec = [(round(lat, 3), round(long, 3))for lat, long in zip(df.latitude, df.longitude)]


In [59]:
len(set(coords))

140336

In [70]:
len(set(coords_2dec)), len(set(coords_3dec))

(11824, 73492)